In [15]:
import dotenv
dotenv.load_dotenv()
import os
MYSQL_USER=os.getenv('MYSQL_USER')
MYSQL_PASSWORD=os.getenv('MYSQL_PASSWORD')

In [16]:
# print(MYSQL_USER)
# print(MYSQL_PASSWORD)

In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from sqlalchemy import create_engine
import pymysql

In [ ]:
mysql_key_str=os.environ.get("MYSQL_KEY")

In [ ]:
mysql_key_str

In [ ]:
engine = create_engine("mysql+pymysql://root:" + os.environ.get("mysql_key") + '@localhost:3306/airflow_project')
# engine = create_engine("mysql+pymysql://root:" + '@localhost:3306/airflow_project')

In [ ]:
abb_vis_df = pd.read_sql('select * from airflow_project.nyc_abb', con=engine)
abb_vis_df.head()

#### more detail data cleaning

In [ ]:
abb_vis_df.loc[abb_vis_df['reviews_per_month'].isnull(), 'reviews_per_month'] = 0

In [ ]:
abb_vis_df = abb_vis_df.loc[:, ~ abb_vis_df.columns.str.contains('^Unnamed')]

In [ ]:
abb_vis_df = abb_vis_df[abb_vis_df['price']>0]

In [ ]:
abb_vis_df = abb_vis_df[abb_vis_df['minimum_nights']<=365]

In [ ]:
abb_vis_df = abb_vis_df.set_index('host_id')

In [ ]:
abb_vis_df.drop(columns=['index']).head()

In [ ]:
abb_vis_df.shape

In [ ]:
abb_vis_df.dtypes

In [ ]:
abb_vis_df.isnull().sum()

### Example using SQL query to pull out data

In [ ]:
# More Visualization from SQL query
connection = engine.connect()

In [ ]:
# Average Price of Airbnb in NYC /night
avg_abb_price = connection.execute('SELECT AVG(price) FROM airflow_project.nyc_abb')

In [ ]:
data_avg_abb_price = [row for row in avg_abb_price]
df_avg_abb_price = pd.DataFrame.from_records(data_avg_abb_price)
df_avg_abb_price.columns = ['Average price/night of Airbnb in NYC 2019']
df_avg_abb_price

In [ ]:
# Average Price of Airbnb in NYC /night (Manhattan) 
avg_abb_price_M = connection.execute('SELECT AVG(price) \
                                     FROM airflow_project.nyc_abb \
                                     WHERE borough = "Manhattan"')

In [ ]:
data_avg_abb_price_M = [row for row in avg_abb_price_M]
df_avg_abb_price_M = pd.DataFrame.from_records(data_avg_abb_price_M)
df_avg_abb_price_M.columns = ['Average price/night of Airbnb in NYC 2019 (Manhattan)']
df_avg_abb_price_M

In [ ]:
# Average Price of Airbnb in NYC /night (Queens) 
avg_abb_price_Q = connection.execute('SELECT AVG(price) \
                                     FROM airflow_project.nyc_abb\
                                     WHERE borough = "Queens"')

In [ ]:
data_avg_abb_price_Q = [row for row in avg_abb_price_Q]
df_avg_abb_price_Q = pd.DataFrame.from_records(data_avg_abb_price_Q)
df_avg_abb_price_Q.columns = ['Average price/night of Airbnb in NYC 2019 (Queens)']
df_avg_abb_price_Q

In [ ]:
#Total Number of review
sum_abb_reviews = connection.execute('SELECT SUM(number_of_reviews)\
                                        FROM airflow_project.nyc_abb')

In [ ]:
data_sum_abb_reviews = [row for row in sum_abb_reviews]
df_sum_abb_reviews = pd.DataFrame.from_records(data_sum_abb_reviews)
df_sum_abb_reviews.columns = ['Total nunber of reviews for Airbnb housing in NYC 2019']
df_sum_abb_reviews

### Next, we will look at the price distribution by the different boroughs

In [ ]:
abb_vis_df.borough.unique()

In [ ]:
set(abb_vis_df['borough'])

In [ ]:
abb_vis_df.groupby('borough')['price'].describe().round(2)

In [ ]:
#The minimum nights for each borough
abb_vis_df.groupby('borough')['minimum_nights'].describe()

In [ ]:
#Pivot table, 5 boroughs
abb_vis_df_pt = abb_vis_df.pivot_table(index='borough', columns='room_type', values='price', aggfunc='mean')
abb_vis_df_pt.round(2)

### Now, let's do simle combing dataset set by couting data from 5 boroughs

#### NYC Airbnb Housing Data Count Summary

In [ ]:
abb_borough_count = abb_vis_df.groupby('borough').borough.count()
abb_borough_count

#### NYC Park Data Count Summary

In [ ]:
nyc_park_vis_df = pd.read_sql('select * from airflow_project.nyc_park', con=engine)
nyc_park_vis_df.head()

In [ ]:
#Rename  B = Broonlyn, X = Bronx, M = Manhattan,  Q = Queens,R = Staten Island,
nyc_park_vis_df.loc[nyc_park_vis_df.borough == 'B', 'borough'] = 'Brooklyn'
nyc_park_vis_df.loc[nyc_park_vis_df.borough == 'X', 'borough'] = 'Bronx'
nyc_park_vis_df.loc[nyc_park_vis_df.borough == 'M', 'borough'] = 'Manhattan'
nyc_park_vis_df.loc[nyc_park_vis_df.borough == 'Q', 'borough'] = 'Queens'
nyc_park_vis_df.loc[nyc_park_vis_df.borough == 'R', 'borough'] = 'Staten Island'

In [ ]:
nyc_park_vis_df.head()

In [ ]:
park_borough_count = nyc_park_vis_df.groupby('borough').borough.count()
park_borough_count

#### NYC Hot Spot Data Count Summary

In [ ]:
nyc_hot_spot_df = pd.read_sql('select * from airflow_project.nyc_hot_spot', con=engine)
nyc_hot_spot_df.head()

In [ ]:
hot_spot_borough_count = nyc_hot_spot_df.groupby('borough').borough.count()
hot_spot_borough_count

#### NYC Hotel  Data Count Summary

In [ ]:
nyc_hotel_vis_df = pd.read_sql('select * from airflow_project.nyc_hotel', con=engine)
nyc_hotel_vis_df.head()

In [ ]:
nyc_hotel_vis_df.loc[nyc_hotel_vis_df.borough == 'BROOKLYN', 'borough'] = 'Brooklyn'
nyc_hotel_vis_df.loc[nyc_hotel_vis_df.borough == 'BRONX', 'borough'] = 'Bronx'
nyc_hotel_vis_df.loc[nyc_hotel_vis_df.borough == 'MANHATTAN', 'borough'] = 'Manhattan'
nyc_hotel_vis_df.loc[nyc_hotel_vis_df.borough == 'QUEENS', 'borough'] = 'Queens'
nyc_hotel_vis_df.loc[nyc_hotel_vis_df.borough == 'STATEN IS', 'borough'] = 'Staten Island'

In [ ]:
nyc_hotel_vis_df.head()

In [ ]:
hotel_borough_count = nyc_hotel_vis_df.groupby('borough').borough.count()
hotel_borough_count

#### NYC Shooting Data Count Summary

In [ ]:
nyc_shooting_vis_df = pd.read_sql('select * from airflow_project.nyc_shooting', con=engine)
nyc_shooting_vis_df.head()

In [ ]:
nyc_shooting_vis_df.loc[nyc_shooting_vis_df.borough == 'BROOKLYN', 'borough'] = 'Brooklyn'
nyc_shooting_vis_df.loc[nyc_shooting_vis_df.borough == 'BRONX', 'borough'] = 'Bronx'
nyc_shooting_vis_df.loc[nyc_shooting_vis_df.borough == 'MANHATTAN', 'borough'] = 'Manhattan'
nyc_shooting_vis_df.loc[nyc_shooting_vis_df.borough == 'QUEENS', 'borough'] = 'Queens'
nyc_shooting_vis_df.loc[nyc_shooting_vis_df.borough == 'STATEN ISLAND', 'borough'] = 'Staten Island'

In [ ]:
shooting_borough_count = nyc_shooting_vis_df.groupby('borough').borough.count()
shooting_borough_count

#### NYC Publica Housing Data Count Summary

In [ ]:
pb_houing_vis_df = pd.read_sql('select * from airflow_project.nyc_housing', con=engine)
pb_houing_vis_df.head()

In [ ]:
pb_houing_borough_count = pb_houing_vis_df.groupby('borough').borough.count()
pb_houing_borough_count

In [ ]:
count_sum_df = pd.concat([abb_borough_count, park_borough_count,hot_spot_borough_count,
                    hotel_borough_count,shooting_borough_count,pb_houing_borough_count], axis = 1 )

In [ ]:
count_sum_df.columns= ['airbnb', 'park', 'hot spot', 'hotel', 'shooting', 'public housing']

In [ ]:
count_sum_df

In [ ]:
count_sum_df.drop('airbnb', axis=1).plot(kind = 'barh', legend = True, style = 'ggplot',figsize=(15,5), fontsize= 20, title='NYC Open Data Distribution in 5 Boroughs')

In [ ]:
# subplot
import matplotlib as mpl

fig,ax=plt.subplots(3,2, figsize=(14, 10), constrained_layout=True)


ax[0,0].pie(count_sum_df['airbnb'], labels = ['Bronx', 'Brooklyn', "Manhattan", 'Queens', 'Staten Island'], 
            textprops={'fontsize': 18})
ax[0,0].set_title('NYC - Airbnb')


ax[0,1].pie(count_sum_df['park'], labels = ['Bronx', 'Brooklyn', "Manhattan", 'Queens', 'Staten Island'],
           textprops={'fontsize': 18})
ax[0,1].set_title('NYC - Park')

ax[1,0].pie(count_sum_df['hot spot'], labels = ['Bronx', 'Brooklyn', "Manhattan", 'Queens', 'Staten Island'],
           textprops={'fontsize': 18})
ax[1,0].set_title('NYC - Hot Spot')

ax[1,1].pie(count_sum_df['hotel'], labels = ['Bronx', 'Brooklyn', "Manhattan", 'Queens', 'Staten Island'],
           textprops={'fontsize': 18})
ax[1,1].set_title('NYC - Hotel')

ax[2,0].pie(count_sum_df['shooting'], labels = ['Bronx', 'Brooklyn', "Manhattan", 'Queens', 'Staten Island'],
           textprops={'fontsize': 18})
ax[2,0].set_title('NYC - Shooting')

ax[2,1].pie(count_sum_df['public housing'], labels = ['Bronx', 'Brooklyn', "Manhattan", 'Queens', 'Staten Island'],
           textprops={'fontsize': 18})
ax[2,1].set_title('NYC - Public Housing')

#### Data visulization

In [ ]:
abb_vis_df_pt.plot(kind = 'barh', legend = True, style = 'ggplot',figsize=(15,5), fontsize= 20, title='Price of room type in 5 boroughs')

In [ ]:
#All houses get few reviews per month
sns.distplot(abb_vis_df['reviews_per_month'])
plt.ylabel('number_of_records')
plt.show()

In [ ]:
# Airbnb Location by Borough Vis
sns.catplot('borough', data=abb_vis_df, kind = 'count', height =9, aspect = 1)

In [ ]:
sns.catplot('room_type', data=abb_vis_df, kind = 'count', height =8, aspect = 1)

In [ ]:
#The most popular neigbourhood
data = abb_vis_df.neighbourhood.value_counts()[:10]
plt.figure(figsize=(10,6))
x = list(data.index)
y = list(data.values)
x.reverse()
y.reverse()

plt.title("Most popular neighbourhood")
plt.xlabel("Number of guest whom host in this area")
plt.ylabel("Neighbourhood area")

plt.barh(x, y, color = 'green')


In [ ]:
# Corrlation matric
sns.set(font_scale =2)
plt.figure(figsize =(20, 10))
sns.heatmap(abb_vis_df.corr(), annot=True)

In [ ]:
#All other feature correlation with price
abb_vis_df.drop('price', axis=1).corrwith(abb_vis_df.price).plot.barh(figsize=(10, 8),
                                    title='Correlation with Response Variable', fontsize=15, grid=True)